## Helper file to generate Configs efficiently

In [53]:
import os

def variable_combinations(cls):

        def enumerate_vars(mega, count):

            if count > len(mega)-1:
                return [[]]
            elif not isinstance(getattr(cls, mega[count]), list):
                res =  [
                    [(mega[count], getattr(cls, mega[count]))] + e
                    for e in enumerate_vars(mega, count+1)
                ]
                return res
            else:
                attr = getattr(cls, mega[count])
                downstream = enumerate_vars(mega, count+1)
                results = []
                for val in attr:
                    for d in downstream:
                        results.append([(mega[count], val)] + d)
                return results
                    
        list_of_vars = [v for v in dir(cls) if not callable(getattr(cls, v)) and not v.startswith('__')]
        return enumerate_vars(list_of_vars, count=0)

def write_config(file_name: str, template: str, vars: list):
    for v in vars:
        template = template.replace(v[0], str(v[1]))
    assert not os.path.exists(file_name), f"file {file_name} exists. please change the file_name to avoid accidental overwriting."
    with open(file_name, 'w') as fs:
        fs.write(template)

In [107]:
INDEX = 34
class Variables:
    BASE = '../' 
    DATA_FILE = f'\"{BASE}dataset/cityscapes.py\"'
    RUNTIME_FILE = f'\"{BASE}../configs/_base_/default_runtime.py\"'
    MODEL_FILE = f'\"{BASE}../configs/_base_/models/fpn_r50.py\"'
    SPG = 1
    GPU = 8
    QUERY_EPOCH = 5
    BUDGET = 'int(256*512*0.005) * 2975'
    SAMPLE_ROUNDS = 10
    HEURISTIC = '\"sparsity\"'
    VIZ_SIZE = 10
    VIZ_DIR = f'\"viz_tune_sparsity_{INDEX}\"'
    WANDB_NAME = f'\"fpnR50_gtav_sparsity_{INDEX}\"'
    INIT_LR = "0.001"
    MIN_LR = "5e-5"
    INFLECTION = "0.16"
    K = 2
    LOAD_FROM = '\"experiments/gtav_ckpt_fpnR50.pth\"'
    SAMPLE_EVENLY = False

In [108]:
template_path = "./config_template.txt"
write_folder = "./tune_sparsity/"
with open(template_path, 'r') as fs:
    contents = fs.read()

In [109]:
x = variable_combinations(cls=Variables)
write_config(file_name=os.path.join(write_folder, f"sparsity_{INDEX}.py"), template=contents, vars=x[0])